# PDPBioGen Validation — NV vs Classical (Simulated)\n\nThis notebook runs the simulation protocol and produces benchmark figures.

In [ ]:
from pdpbiogen.simulator import generate_phantom_trials, matched_filter_scores\nfrom pdpbiogen.sensors.nv_sim import NVSimulatedSensor\nfrom pdpbiogen.sensors.classical_sim import ClassicalSimulatedSensor\nfrom pdpbiogen.analysis import compute_snr, compute_auc, bootstrap_auc_diff\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generate trials (simulation)\ntrials, labels = generate_phantom_trials(fs=5000, amplitudes=[0.1,0.2,0.5,1.0], n_trials=40)\n\n# Simulated sensors\nnv = NVSimulatedSensor(fs=5000, noise_pT_per_sqrtHz=5.0)\ncl = ClassicalSimulatedSensor(fs=5000, noise_pT_per_sqrtHz=25.0)\n\n# Record\nwindows_nv = nv.record_trials(trials)\nwindows_cl = cl.record_trials(trials)\n\n# derive template (average of signal trials)\nsignal_idxs = np.where(labels==1)[0]\ntemplate = np.mean(windows_nv[signal_idxs], axis=0)\n\n# scores\nscores_nv = matched_filter_scores(windows_nv, template)\nscores_cl = matched_filter_scores(windows_cl, template)\n\n# compute AUC\nauc_nv = compute_auc(scores_nv, labels)\nauc_cl = compute_auc(scores_cl, labels)\nprint('AUC NV', auc_nv, 'AUC Classical', auc_cl)\n\n# bootstrap diff\nmean_diff, ci, pval = bootstrap_auc_diff(scores_nv, scores_cl, labels, nboot=500)\nprint('AUC diff mean', mean_diff, 'CI', ci, 'p', pval)\n\n# SNR\npre = int(5000*0.05)\nsnrs_nv = compute_snr(windows_nv, pre)\nsnrs_cl = compute_snr(windows_cl, pre)\nprint('Mean SNR NV (dB)', np.mean(snrs_nv), 'Classical', np.mean(snrs_cl))\n\n# Plot sample\nplt.figure(figsize=(8,6))\nplt.subplot(3,1,1)\nplt.plot(windows_nv[0], label='NV sample')\nplt.legend()\nplt.subplot(3,1,2)\nplt.plot(windows_cl[0], label='Classical sample')\nplt.legend()\nplt.subplot(3,1,3)\nplt.hist(scores_nv, alpha=0.6, label='NV')\nplt.hist(scores_cl, alpha=0.6, label='Classical')\nplt.legend()\nplt.tight_layout()\nplt.show()